In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from torch.utils.data import DataLoader
import random
import pandas as pd
import torch
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, log_loss

import gpl


/home/aljaz/miniforge3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-23 09:18:24 - PyTorch version 2.3.0 available.
2024-05-23 09:18:24 - Loading faiss with AVX2 support.
2024-05-23 09:18:24 - Successfully loaded faiss with AVX2 support.
[2024-05-23 09:18:24] INFO [gpl.train.train:81] Corpus does not exist in ./cord_data. Now clone the one from the evaluation path None


AssertionError: No corpus found in evaluation path None! It should be in the BeIR format. For more details, please refer to https://github.com/UKPLab/beir#beers-available-datasets.

In [2]:
testPath = "/home/aljaz/FAKS/ul-fri-nlp-course-project-sneguljcica/data/raw/Annotation_sentence-level_utf8.txt"
df = pd.read_csv(testPath, sep="\t")
df = df.iloc[:, [3, 12]]

df = df.iloc[:38000, :]

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0], df.iloc[:, 1], train_size=0.8, random_state=42)

In [3]:

modelss= ["bert-base-multilingual-uncased", "hate_speech_slo", "all-MiniLM-L12-v2" ]

base_model_path = "/home/aljaz/FAKS/ul-fri-nlp-course-project-sneguljcica/models/models100/"

d = {}

for model_name in modelss:
    #base model testing

    bbert = models.Transformer(base_model_path + model_name)
    bpooling = models.Pooling(bbert.get_word_embedding_dimension(), 'cls')
    model = SentenceTransformer(modules=[bbert, bpooling])

    print("Model: ", model_name)
    embedded_train = model.encode(X_train.to_numpy())
    embedded_test = model.encode(X_test.to_numpy())

    lr = LogisticRegression(max_iter=1000)
    lr.fit(embedded_train, y_train.to_numpy())

    y_pred = lr.predict(embedded_test)
    y_pred_probabs = lr.predict_proba(embedded_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    logloss = log_loss(y_test, y_pred_probabs)
    print("F1 score: ", f1)
    print("Log loss: ", logloss)
    d[model_name] = {"F1" :f1,"Log-Loss": logloss}

    torch.cuda.empty_cache() 

    # tsdae model

    bert = models.Transformer(base_model_path + "tsdae-" + model_name)
    pooling = models.Pooling(bert.get_word_embedding_dimension(), 'cls')
    model = SentenceTransformer(modules=[bert, pooling])

    print("Model: ", "tsdae-" + model_name)
    embedded_train = model.encode(X_train.to_numpy())
    embedded_test = model.encode(X_test.to_numpy())

    lr = LogisticRegression(max_iter=1000)
    lr.fit(embedded_train, y_train.to_numpy())

    y_pred = lr.predict(embedded_test)
    y_pred_probabs = lr.predict_proba(embedded_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    logloss = log_loss(y_test, y_pred_probabs)
    print("F1 score: ", f1)
    print("Log loss: ", logloss)
    d["tsdae-" + model_name] = {"F1" :f1,"Log-Loss": logloss}

    torch.cuda.empty_cache()


scores = pd.DataFrame(d).T
print(scores)
scores.to_csv("scores.csv")


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 